In [1]:
 import os
 import shutil

 # Input folder containing the images
 input_dir = r"/kaggle/input/skin-disease-dataset/dataset/train"
 # Output folder for renamed images
 output_dir = r"/kaggle/working/renamed_train"

 # Ensure the output directory exists
 os.makedirs(output_dir, exist_ok=True)

 # Dictionary to track counts for each class
 class_counts = {}

 # Traverse through each subdirectory
 for root, dirs, files in os.walk(input_dir):
     for file_name in files:
         # Full path of the image
         img_path = os.path.join(root, file_name)

         # Skip non-image files
         if not file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
             print(f"Skipping non-image file: {file_name}")
             continue

         # Get the folder name (class name) as the class identifier
         class_name = os.path.basename(root)

         # Initialize or increment the count for this class
         if class_name not in class_counts:
             class_counts[class_name] = 1
         else:
             class_counts[class_name] += 1

         # Generate new file name in the format ClassName(Count).Extension
         count = class_counts[class_name]
         ext = os.path.splitext(file_name)[1]  # Get file extension
         new_name = f"{class_name}({count}){ext}"
         new_path = os.path.join(output_dir, new_name)

         # Copy and rename the file to the output directory
         shutil.copy(img_path, new_path)

 # Print the total number of images for each class
 print("\nImage counts by class:")
 for class_name, count in class_counts.items():
     print(f"{class_name}: {count} images")

 print("\nRenaming and consolidation complete!")


Image counts by class:
Eczema: 999 images
Melanoma: 1000 images
Basal Cell: 1000 images
Seborrheic: 1000 images
Atopic Dermatitis: 1000 images
Melanocytic: 1000 images
Benign Keratosis: 1201 images
Warts Molluscum: 1000 images
Psoriasis: 1000 images
Tinea Ringworms Candidiasis: 990 images

Renaming and consolidation complete!


In [3]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet152  # Use ResNet152
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import random
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Dense, Dropout
from tensorflow.keras.models import Model

# Define class mapping
class_mapping = {
    "Seborrheic": 0,
    "Melanocytic": 1,
    "Melanoma": 2,
    "Eczema": 3,
    "Basal_Cell": 4,
}

# Preprocess images: resize and normalize
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: {image_path} could not be loaded.")
        return None

    resized_image = cv2.resize(image, (224, 224))  # Resizing for ResNet152
    img_normalized = resized_image.astype('float32') / 255.0  # Normalize to [0, 1]
    return resized_image

def load_data_from_single_folder(folder):
    images, labels = [], []

    # Get a sorted list of image filenames
    image_files = sorted([f for f in os.listdir(folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

    for image_name in image_files:
        image_path = os.path.join(folder, image_name)
        label = image_name.split('(')[0].strip().replace(' ', '_')  # Handle spaces and extract class name
        
        if label in class_mapping:
            label_index = class_mapping[label]
        else:
            continue

        preprocessed_image = preprocess_image(image_path)
        if preprocessed_image is not None:
            images.append(preprocessed_image)
            labels.append(label_index)

    print(f"Loaded {len(images)} images and {len(labels)} labels.")
    return np.array(images), np.array(labels)

# Paths for train and test folders
train_folder = r'/kaggle/working/renamed_train'
test_folder = r"/kaggle/working/renamed_Test"

# Load data
X_train, y_train = load_data_from_single_folder(train_folder)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

# Image Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# === Load ResNet152 Model ===
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling=None)

# Fine-tune ResNet152
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

predictions = Dense(len(class_mapping), activation="softmax")(x)  # Output layer with softmax activation

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Unfreeze the last 20 layers for fine-tuning
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Compile the model with Adam optimizer
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',  # Sparse because labels are integers
              metrics=['accuracy'])

# Learning rate scheduler
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1)
]

# Train the model
epochs = 50
batch_size = 16
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                    validation_data=(X_test, y_test), callbacks=callbacks, shuffle=True)

# Predict on the test data
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)

# Confusion matrix for test set
test_conf_matrix = confusion_matrix(y_test, y_test_pred_classes)
print(f"Test Confusion Matrix:\n{test_conf_matrix}")

# Calculate overall accuracy for the test set
test_accuracy = np.mean(y_test_pred_classes == y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Print sample predictions
label_to_class = {v: k for k, v in class_mapping.items()}
for i in range(10):
    predicted_class = label_to_class[y_test_pred_classes[i]]
    actual_class = label_to_class[y_test[i]]
    print(f"Predicted: {predicted_class}, Actual: {actual_class}")


Loaded 4999 images and 4999 labels.
Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 278s 539ms/step - accuracy: 0.3021 - loss: 1.9686 - val_accuracy: 0.7580 - val_loss: 0.7630 - learning_rate: 1.0000e-05
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 60s 241ms/step - accuracy: 0.6522 - loss: 0.9324 - val_accuracy: 0.8480 - val_loss: 0.5138 - learning_rate: 1.0000e-05
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 60s 239ms/step - accuracy: 0.7584 - loss: 0.7130 - val_accuracy: 0.8610 - val_loss: 0.4542 - learning_rate: 1.0000e-05
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 60s 239ms/step - accuracy: 0.7687 - loss: 0.6348 - val_accuracy: 0.9020 - val_loss: 0.3551 - learning_rate: 1.0000e-05
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 60s 239ms/step - accuracy: 0.8394 - loss: 0.4886 - val_accuracy: 0.9110 - val_loss: 0.3203 - learning_rate: 1.0000e-05
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 60s 239ms/step - accuracy: 0.8576 - loss: 0.4357 - val_accuracy: 0.9200 - val_loss: 0.2974 - learning_rate: 1.0000e-05
Epoch 7/5